In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

/home/philippe/MolGenDocking


/home/philippe/miniconda3/envs/MolGen/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from pathlib import Path
import json
import re

import pandas as pd
import numpy as np
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
import os
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit import DataStructs

from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.metrics import roc_curve, roc_auc_score
from notebooks.utils import PandasTableFormatter
from mol_gen_docking.evaluation.diversity_aware_top_k import diversity_aware_top_k
from notebooks.utils import *
from mol_gen_docking.data.pydantic_dataset import read_jsonl
import jsonlines
from tqdm.auto import tqdm

tqdm.pandas()

FIG_PATH = "MolGenDocking-latex/Figures/Others"
os.makedirs(FIG_PATH, exist_ok=True)


# Generation

In [4]:

import re
def unsafe_parse(output):
    parsed_answer = re.findall(rf"(?:<answer>|<\|answer_start\|>)((?:(?!<answer>|<\|answer_start\|>).)*?)(?:</answer>|<\|answer_end\|>)", output, re.DOTALL)
    if parsed_answer == []:
        return False

    parsed_answer = parsed_answer[-1].split("<|answer_start|>")[-1].split("<answer>")[-1]
    if parsed_answer == "Unsafe question, I can't answer.":
        return True
    else:
        return False

MOLSTRAL_PATH = Path("MolGenOutput/test_ood/ether0")
df = []
for f in tqdm(MOLSTRAL_PATH.iterdir()):
    if not "scored" in f.name:
        continue
    res = []
    with jsonlines.open(f) as reader:
        for line in reader:
            df.append(
                [line["output"], unsafe_parse(line["output"]), line["metadata"]["properties"], line["metadata"]["prompt_id"]]
            )

df = pd.DataFrame(df, columns=["output", "unsafe", "properties", "prompt_id"])
print(f"Generates unsafe completions: {df.unsafe.mean()*100:.1f}%")

0it [00:00, ?it/s]

Generates unsafe completions: 11.1%


In [5]:
df.groupby("unsafe")["prompt_id"].nunique()

unsafe
False    1000
True      983
Name: prompt_id, dtype: int64

In [6]:
example_df = df[df.unsafe].sample(10)
for row in example_df.itertuples():
    print(f"Prompt ID: {row.prompt_id}")
    print(row.output)
    print("-"*80)

Prompt ID: 6623-108
<|think_start|>Okay, let's see. The user is asking about a chemical compound, specifically "User and Assistant," which might be a typo for a chemical structure. The user is asking about a chemical compound, specifically "User and Assistant," which might be a typo for a chemical structure. The user is asking about a chemical compound, specifically "User and Assistant," which might be a typo for a chemical structure. The user is asking about a chemical compound, specifically "User and Assistant," which might be a typo for a chemical structure. The user is asking about a chemical compound, specifically "User and Assistant," which might be a typo for a chemical structure.

The user is asking about a chemical compound, specifically "User and Assistant," which might be a typo for a chemical structure. The user is asking about a chemical compound, specifically "User and Assistant," which might be a typo for a chemical structure. The user is asking about a chemical compound

In [7]:
MOLSTRAL_PATH = Path("MolGenOutput/polaris")
df = []
for f in tqdm(MOLSTRAL_PATH.rglob("*ether0*")):
    if not "scored" in f.name:
        continue
    res = []
    with jsonlines.open(f) as reader:
        for line in reader:
            df.append(
                [line["output"], unsafe_parse(line["output"]), line["metadata"]["properties"], line["metadata"]["prompt_id"]]
            )

df = pd.DataFrame(df, columns=["output", "unsafe", "properties", "prompt_id"])
print(f"Generates unsafe completions: {df.unsafe.mean()*100:.1f}%")

0it [00:00, ?it/s]

Generates unsafe completions: 45.8%


In [10]:
example_df = df[df.unsafe].sample(10)
for row in example_df.itertuples():
    print(f"Prompt ID: {row.prompt_id}")
    print(row.output)
    print("-"*80)

Prompt ID: novartis:novartis-cyp3a4-v1_train_13149
<|think_start|>Okay, so I need to find the log-inactivation rate constant of CYP enzymes for the given molecule. The molecule is O=C(O)c1c(-c2nccc3ccccc23)[nH]c(-c2ccccc2)c1S(=O)(=O)O.

First, parse the SMILES notations. The main part is O=C(O)c1... which suggests a carboxylic acid group connected to a ring. The central ring is c1, with substituents:

- c(-c2nccc3ccccc23): This is a substituent connected via carbon, likely a 2-pyridyl or 1H-indol-3-yl group, with an indole or similar bicyclic ring system (c2nccc3ccccc23).
- [nH]c(-c2ccccc2): A pyrrole-like NH connected to a phenyl group (c2ccccc2).
- S(=O)(=O)O: A sulfonic acid group.
- c1: The central ring.

The main molecule is a carboxylic acid-linked heterocyclic ring with a sulfonyl and a phenyl group. The ring likely includes a thiazole or similar due to the sulfur.

Considering the question: the molecule's log-inactivation rate constant. Typically, log-inactivation rate constant